# Data Preparation

Inhaltsverzeichnis

In [1]:
%load_ext autoreload
%autoreload 2

## Data Cleaning

In der Data Exploration konnte festgestellt werden, dass der Datensatz nicht vollständig unseren Anforderungen entspricht und einige Fehler aufweist. Im Data-Cleaning Prozess soll daher der Datensatz angepasst und verbessert werden. Identifizierte Fehler sollen behoben werden. Ziel ist es, den Datensatz für die Preprocessing Pipeline vorzubereiten. 

#### 0. Datensatz laden

In [2]:
from src import utils
import pandas as pd

# load dataframe
df = utils.load_pkl(path='../data/raw/twitter_tweets_raw.pkl')[0]

#### 1. Duplikate löschen 

In [3]:
df.drop_duplicates(subset=['rawContent'], inplace=True)

# check for success
if df['rawContent'].duplicated().any():
    print(f"{len(df[df['rawContent'].duplicated()])} duplicates found.")

#### 2. Nicht-englische Beiträge löschen 

In [4]:
non_english_posts = df.query('lang != "en"')
df.drop(index=non_english_posts.index, inplace=True)

# check for success
if not df['lang'].eq('en').all():
    print(df.query('lang != "en"'))

#### 3. Date aktualisieren

In [5]:
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)

#### 4. Irrelevante Daten löschen

In [6]:
df.drop(columns=['renderedContent', 'id', 'user', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 
                 'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'links', 'media', 
                 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers', 
                 'coordinates', 'place', 'hashtags', 'cashtags', 'card', 'viewCount', 'vibe'], inplace=True)

#### 5. Gesäuberten Datensatz speichern

In [7]:
df.set_index('url', inplace=True)
df.reset_index(inplace=True)
df.to_feather('../data/intermediate/twitter_tweets_intermediate.feather')

---

## Preprocessing Pipeline

Nachdem der Datensatz gesäubert wurde, werden die Textdaten in eine sogenannte Preprocessing Pipeline gegeben. Die Preprocessing Pipeline ist entscheidend für das spätere Ergebnis des Modelings. Diese stellt sicher, dass der Text für das Modell geeignet ist. Die Pipeline, die für die Anforderungen des Projekts speziell entworfen wurde, kann wie folgt visualisiert werden:

TODO: ABBILDUNG VON DER PREPROCESSING PIPELINE

#### 0. Packages & Datensatz laden 

In [8]:
import pandas as pd
import contractions
import nltk
import string
import emoji
import re

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# download required nltk packages
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# load dataframe
df = pd.read_feather('../data/intermediate/twitter_tweets_intermediate.feather')
df.head(4)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,url,date,rawContent
0,https://twitter.com/YueDongCS/status/164159107...,2023-03-30 23:59:46,My condolences and sad that the #NLP and #AI c...
1,https://twitter.com/Mlearning_ai/status/164159...,2023-03-30 23:59:43,Hiring Now: The Top Jobs of the Future Fueled ...
2,https://twitter.com/HackerAran7/status/1641591...,2023-03-30 23:59:43,What’s the hack. #stem #science #stemeducation...
3,https://twitter.com/Stemble_/status/1641590942...,2023-03-30 23:59:14,"🚀 Mark your calendars, Apple enthusiasts! 🗓️\n..."


#### 1.  URLs entfernen

In [9]:
def remove_urls(text):
    return re.sub(r'http\S+', '', text)

df['_nonurl'] = df['rawContent'].apply(remove_urls)

#### 2. Kontraktionen auflösen

In [10]:
def fix_contractions(text):
    return contractions.fix(text)

df['_nonurl_fixedcontractions'] = df['_nonurl'].apply(fix_contractions)
del df['_nonurl']

#### 3. Tokenization durchführen

In [11]:
# define tokenizer function
tokenize = nltk.tokenize.word_tokenize

df['_nonurl_fixedcontractions_tokenized'] = df['_nonurl_fixedcontractions'].apply(tokenize)
del df['_nonurl_fixedcontractions']

#### 4. Tokens in Kleinbuchstaben umwandeln

In [12]:
def lowercase(tokens):
    return [token.lower() for token in tokens]

df['_nonurl_fixedcontractions_tokenized_lowercase'] = df['_nonurl_fixedcontractions_tokenized'].apply(lowercase)
del df['_nonurl_fixedcontractions_tokenized']

#### 5. Entfernung von Satzzeichen

In [13]:
# adding more characters to the punctuation list
punct = string.punctuation + "’" + "``" +"`" + "''" +"'"

def remove_punct(tokens):
    return [token for token in tokens if token not in punct]

df['_nonurl_fixedcontractions_tokenized_lowercase_nonpunctation'] = df['_nonurl_fixedcontractions_tokenized_lowercase'].apply(remove_punct)
del df['_nonurl_fixedcontractions_tokenized_lowercase']

#### 6.  Stopwörter entfernen

In [14]:
# define list of stopwords
stop_words = stopwords.words('english')

def remove_stopwords(tokens):
    return [token for token in tokens if token not in stop_words]

df['_nonurl_fixedcontractions_tokenized_lowercase_nonpunctation_nonstopwords'] = df['_nonurl_fixedcontractions_tokenized_lowercase_nonpunctation'].apply(remove_stopwords)
del df['_nonurl_fixedcontractions_tokenized_lowercase_nonpunctation']

#### 7.  Emojis entfernen

In [15]:
def remove_emoji(tokens):
    return [token for token in tokens if not any(char in emoji.EMOJI_DATA for char in token)]
    
df['_nonurl_fixedcontractions_tokenized_lowercase_nonpunctation_nonstopwords_nonemoji'] = df['_nonurl_fixedcontractions_tokenized_lowercase_nonpunctation_nonstopwords'].apply(remove_emoji)
del df['_nonurl_fixedcontractions_tokenized_lowercase_nonpunctation_nonstopwords']

#### 8. Lemmatisierung durchführen

In [16]:
# initialization of the lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

df['preprocessed_text'] = df['_nonurl_fixedcontractions_tokenized_lowercase_nonpunctation_nonstopwords_nonemoji'].apply(lemmatize)
del df['_nonurl_fixedcontractions_tokenized_lowercase_nonpunctation_nonstopwords_nonemoji']

#### 9. Preprocessed Datensatz speichern

In [17]:
df.to_feather('../data/processed/twitter_tweets_processed.feather')
df.to_csv('../data/processed/twitter_tweets_processed.csv', index=False)

---